In [1]:
import os
import sys
import scipy
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
import scipy.io as sio
import anndata as ad
import matplotlib.pyplot as plt

os.chdir("/data/wuqinhua/phase/age")

# 1. UMAP

In [2]:
attnData = pd.read_csv("./Analysis_result/Attn_result/attn_age_cell_PHASE.csv")

In [ ]:
sc.settings.verbosity = 1
sc.settings.figdir = './Plot/Attn_plot'
sc.settings.set_figure_params(dpi=100, fontsize=10, dpi_save=400,
    facecolor = 'white', figsize=(6,6), format='png')

In [3]:
idList = attnData['Tube_id'].unique()
for id in idList:
    attnTmp = attnData[attnData['Tube_id'] == id]
    avgScore = 1 / len(attnTmp)
    log_attn = np.log2(attnTmp['attn'] / avgScore)
    attn_scaled = (log_attn - np.mean(log_attn)) / np.std(log_attn)
    attn_scaled_clipped = np.clip(attn_scaled, -1, 1)
    attnData.loc[attnData['Tube_id'] == id, 'attn_scaled'] = attn_scaled_clipped

In [4]:
adata = ad.read_h5ad('./all_pbmc_anno_s.h5ad')
adata.obs["attn_scaled"] = attnData["attn_scaled"].values
adata1 = adata[adata.obs['Age_group'] == "A"]
adata2 = adata[adata.obs['Age_group'] == "B"]
adata3 = adata[adata.obs['Age_group'] == "C"]
adata4 = adata[adata.obs['Age_group'] == "D"]
adata5 = adata[adata.obs['Age_group'] == "E"]

AnnData object with n_obs × n_vars = 2540586 × 5000
    obs: 'batch', 'sample_id', 'group', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_MT', 'pct_counts_MT', 'leiden', 'predicted_labels', 'over_clustering', 'majority_voting'
    var: 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection', 'mean', 'std'
    uns: 'hvg', 'leiden', 'leiden_colors', 'log1p', 'majority_voting_colors', 'neighbors', 'pca', 'predicted_labels_colors', 'sample_id_colors', 'umap'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

### 1.1 UMAP of celltype

In [ ]:
leiden_umap = sc.pl.umap(adata, color=['celltype'],
    show=False,  palette=sns.color_palette("husl", 24),
legend_fontsize=6, frameon=True, title='celltype', save = "_celltype.pdf")

### 1.2 UMAP of group

In [ ]:
leiden_umap = sc.pl.umap(adata, color='Age', show=False, legend_fontsize=6, color_map ='viridis',
                           frameon= True, title='UMAP of Age',save="_Age.pdf") 

### 1.3 UMAP of age group

In [ ]:
leiden_umap = sc.pl.umap(adata, color='Age_group', show=False, legend_fontsize=6, palette =["#FFCCCC", "#999933", "#B0E57C",  "#99CCFF",  "#D2B5E1"],
                           frameon= True, title='UMAP of Age_group',save="_Age_group.pdf") 

### 1.4 UMAP of Attention score

In [6]:
leiden_umap = sc.pl.umap(adata5, color='attn_scaled', show=False, legend_fontsize=6, color_map ='viridis',
                           frameon= True, title='Attention Score of A',save="_attn_Age_A.pdf") 
leiden_umap = sc.pl.umap(adata5, color='attn_scaled', show=False, legend_fontsize=6, color_map ='viridis',
                           frameon= True, title='Attention Score of B',save="_attn_Age_B.pdf") 
leiden_umap = sc.pl.umap(adata5, color='attn_scaled', show=False, legend_fontsize=6, color_map ='viridis',
                           frameon= True, title='Attention Score of C',save="_attn_Age_C.pdf") 
leiden_umap = sc.pl.umap(adata5, color='attn_scaled', show=False, legend_fontsize=6, color_map ='viridis',
                           frameon= True, title='Attention Score of D',save="_attn_Age_D.pdf") 
leiden_umap = sc.pl.umap(adata5, color='attn_scaled', show=False, legend_fontsize=6, color_map ='viridis',
                           frameon= True, title='Attention Score of E',save="_attn_Age_E.pdf") 

# 2. RMSE

In [ ]:
setwd('/home/wuqinhua/Project/PHASE/Age')
rm(list = ls())
gc()

library(tidyr)
library(ggplot2)
library(forestploter)
library(gridExtra)
library(tidyverse)
library(dplyr)
library(broom)
library(ggpubr)
library(randomForest)
library(mice)
library(reshape2)
library(Metrics)

In [ ]:
attnData = read.csv('./Analysis_result/Attn_result/attn_age_cell_PHASE.csv')
head(attnData)
colnames(attnData)

nameAll = unique(attnData$Cluster_names)
nameAll = sort(nameAll)
nameList = nameAll

sampleFold = data.frame(id = character(), celltype = character(),
                        freq = numeric(), attn = numeric())

idList = unique(attnData$Tube_id)
for (id in idList) {
  attnTmp = attnData %>% filter(Tube_id == id)
  avgScore = 1 / dim(attnTmp)[1]
  foldRes = attnTmp %>% group_by(Cluster_names) %>% summarise(res = sum(attn))
  freqRes = attnTmp %>% group_by(Cluster_names) %>% summarise(res = avgScore * n())
  dataTmp = data.frame(id = rep(id,dim(foldRes)[1]),
                       celltype = foldRes$Cluster_names,
                       freq = freqRes$res,
                       attn = foldRes$res)
  dataTmp_s = dataTmp %>% filter(celltype %in% nameList)
  sampleFold = rbind(sampleFold,dataTmp_s)
}
head(sampleFold)

sampleInfo = read.csv('./Info/sample_info.csv')
rownames(sampleInfo) = sampleInfo$Tube_id
sampleInfo_s = sampleInfo[sampleFold$id,]
sampleFold$group = sampleInfo_s$Age_group


In [ ]:
### get the patient-level rmse
sampleFold_cor = sampleFold %>% group_by(id) %>% summarise(res = rmse(freq,attn)) %>% data.frame()
sampleFold_cor$age = sampleInfo[sampleFold_cor$id,'Age']
sampleFold_cor$group = sampleInfo[sampleFold_cor$id,'Age_group']
head(sampleFold_cor)

cor(sampleFold_cor$res,sampleFold_cor$age)

### RMSE and age
p = ggplot(sampleFold_cor, aes(x = age, y = res)) +
  geom_point(color = "#1f77b4", size = 3, alpha = 0.6) +  
  geom_smooth(method = "lm", formula = y ~ poly(x, 2), color = "gray0", size = 1, se = TRUE) +
  labs(
    title = "Age vs. Rmse",
    x = "Age",
    y = "Rmse"
  ) +
  theme_classic() +  
  theme(
    plot.title = element_text(hjust = 0.5, face = "bold"),  
    legend.position = "bottom" 
  )
ggsave("./Plot/Attn_plot/Sum_scatter.pdf", p, width = 6, height = 4)

### RMSE and age group
p <- ggplot(sampleFold_cor,aes(x = group,y = res,fill = group)) +
  geom_boxplot(alpha = 0.6,outlier.size = 0.5) +  # 设置透明度为0.6
  labs(title = "Rmse for each group",
       x = "Age Group", y = "Rmse") +
  theme_classic() +
  theme(panel.grid = element_blank(),
        axis.line = element_line(color = "black"),
        strip.background = element_blank(),
        strip.text = element_text(size = 10)) 
p
ggsave("./Plot/plot_0914/Sum_boxplot.pdf", p, width = 6, height = 4)

In [ ]:
### get the cellype-level rmse
t11 = sampleFold %>% group_by(celltype) %>% summarise(res = rmse(freq,attn)) %>% data.frame()
x1 = t11 %>% arrange(desc(res))
x1$celltype = factor(x1$celltype,levels = x1$celltype)

### RMSE and Cluster
x1 <- x1 %>%
  mutate(celltype = factor(celltype, levels = unique(celltype[order(res, decreasing = F)])))
red_gradient <- colorRampPalette(c("#FADADD", "#E32636", "#8B0000"))

p <- ggplot(x1,aes(x = celltype,y=res,fill=res)) + 
  geom_bar(stat='identity', width=0.6) +
  scale_fill_gradientn(colors=red_gradient(100)) + 
  labs(x="", y='RMSE(freq,attn)', title="RMSE of Cluster") +
  coord_flip() +
  theme_classic() +
  theme(axis.text=element_text(size=10, face="bold"),
        axis.title=element_text(size=12, face="bold"))

ggsave("/home/wuqinhua/Project/PHASE/Age/Plot/plot_0914/Sum_barplot.pdf", p, width =5, height = 3)

# 3. R2

In [ ]:
attnData = read.csv('./Analysis_result/Attn_result/attn_age_cell_PHASE.csv')
head(attnData)
colnames(attnData)

nameAll = unique(attnData$celltype)
nameAll = sort(nameAll)

# cell type
cell_types <- list(
  B = nameAll[1:9],
  CD4 = nameAll[10:25],
  CD8 = nameAll[26:37],
  GD = nameAll[39:43],
  Myeloid = nameAll[45:48],
  NK = nameAll[49:54]
)

plot_list <- list()

for (cell_type in names(cell_types)) {
  
  nameList <- cell_types[[cell_type]]
  
  sampleFold <- data.frame(id = character(), celltype = character(), fold = numeric())
  idList <- unique(attnData$Tube_id)
  
  for (id in idList) {
    attnTmp <- attnData %>% filter(Tube_id == id)
    avgScore <- 1 / dim(attnTmp)[1]
    foldRes <- attnTmp %>% group_by(celltype) %>% summarise(res = median(log2(attn / avgScore)))
    dataTmp <- data.frame(id = rep(id, dim(foldRes)[1]), celltype = foldRes$celltype, fold = foldRes$res)
    dataTmp_s <- dataTmp %>% filter(celltype %in% nameList)
    dataTmp_s$fold <- scale(dataTmp_s$fold)
    
    sampleFold <- rbind(sampleFold, dataTmp_s)
  }
  
  sampleFold.Table <- dcast(sampleFold, id ~ celltype)
  rownames(sampleFold.Table) <- sampleFold.Table$id
  sampleFold.Table$id <- NULL
  sampleInfo <- sampleInfo[rownames(sampleFold.Table), ]
  
  corData <- data.frame(id = character(), cor = numeric(), r2 = numeric(), pvalue = numeric(), pv2 = numeric())
  for (id in nameList) {
    t1 <- data.frame(value = sampleFold.Table[, id], age = sampleInfo$Age, group = sampleInfo$Age_group)
    t11 <- na.omit(t1)
    res <- cor(t11$value, t11$age)
    r2 <- summary(lm(t11$value ~ t11$age))$r.squared
    pvalue <- cor.test(t11$value, t11$age)$p.value
    pv2 <- kruskal.test(value ~ group, data = t11)$p.value
    dataTmp <- data.frame(id = id, cor = res, r2 = r2, pvalue = pvalue, pv2 = pv2)
    corData <- rbind(corData, dataTmp)
  }
  
  corData$abs <- abs(corData$cor)
  corData <- corData %>% mutate(id = as.factor(id), id = fct_reorder(id, r2))
  red_gradient <- colorRampPalette(c("#FADADD", "#E32636", "#8B0000"))
  
  p <- ggplot(corData, aes(x = id, y = r2, fill = r2)) + 
    geom_bar(stat = 'identity', width = 0.6) +
    scale_fill_gradientn(colors = red_gradient(100)) +
    labs(x = "", y = 'r2', title = paste("r2 of", cell_type, "cell")) +
    coord_flip() +
    theme_classic() +
    theme(axis.text = element_text(size = 10, face = "bold"),
          axis.title = element_text(size = 12, face = "bold"))
  
  plot_list[[cell_type]] <- p
}

pdf("./Plot/Attn_plot/barplot_r2.pdf", width = 20, height = 12)
grid.arrange(grobs = plot_list, ncol = 1)
dev.off()

# 4. Boxplot of Attention score

In [ ]:
attnData = read.csv('./Analysis_result/Attn_result/attn_age_cell_PHASE.csv')
head(attnData)
colnames(attnData)

nameAll = unique(attnData$celltype)
nameAll = sort(nameAll)
nameAll

nameList = nameAll

print(nameList)

sampleFold = data.frame(id = character(), celltype = character(), fold = numeric())
idList = unique(attnData$Tube_id)
for (id in idList) {
  attnTmp = attnData %>% filter(Tube_id == id)
  avgScore = 1 / dim(attnTmp)[1]
  foldRes = attnTmp %>% group_by(celltype) %>% summarise(res = median(log2(attn/avgScore)))
  dataTmp = data.frame(id = rep(id,dim(foldRes)[1]),
                       celltype = foldRes$celltype,
                       fold = foldRes$res)
  dataTmp_s = dataTmp %>% filter(celltype %in% nameList)
  dataTmp_s$fold = scale(dataTmp_s$fold)

  sampleFold = rbind(sampleFold,dataTmp_s)
}

sampleFold.Table = dcast(sampleFold,id ~ celltype)
rownames(sampleFold.Table) = sampleFold.Table$id
sampleFold.Table$id = NULL
print(colnames(sampleFold.Table))

sampleInfo = read.csv('.Info/sample_info.csv')
rownames(sampleInfo) = sampleInfo$Tube_id
sampleInfo = sampleInfo[rownames(sampleFold.Table),]
head(sampleFold.Table)

In [ ]:
cell_types <- colnames(sampleFold.Table)
combined_data <- data.frame()

p_values <- data.frame()

for (cell in cell_types) {
  dataTmp <- data.frame(atten = sampleFold.Table[[cell]],
                        group = sampleInfo$Age_group,
                        age = sampleInfo$Age,
                        cell_type = cell)
  dataTmp_s <- na.omit(dataTmp)
  combined_data <- rbind(combined_data, dataTmp_s)
  
  # Kruskal-Wallis
  kw_test <- kruskal.test(atten ~ group, data = dataTmp_s)
  p_values <- rbind(p_values, data.frame(cell_type = cell, p_value = kw_test$p.value))
}

combined_data <- merge(combined_data, p_values, by = "cell_type")

box_plot <- ggplot(combined_data, aes(x = group, y = atten, fill = group)) +
  geom_boxplot(alpha = 0.6, outlier.size = 0.5) +
  labs(title = "Boxplot: Atten by Age Group for All Cell Types",
       x = "Age Group", y = "Atten") +
  theme_classic() +
  theme(panel.grid = element_blank(),
        axis.line = element_line(color = "black"),
        strip.background = element_blank(),
        strip.text = element_text(size = 10)) +
  facet_wrap(~cell_type, scales = "free") +
  geom_text(data = p_values, aes(x = 1.5, y = Inf, label = paste0("p = ", round(p_value, 3))),
            vjust = 1.5, inherit.aes = FALSE)

ggsave("./Plot/Attn_plot/boxplot_all.png", box_plot, width = 15, height = 12)